In [4]:
! pip install spotipy
! pip install python-dotenv

In [1]:
# Import the required modules
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
from dotenv import load_dotenv
import json

In [40]:
load_dotenv(dotenv_path=".dotenv") # load environment variables from .env file

# access the variables using os.environ.get
client_id = os.environ.get("client_id")
client_secret = os.environ.get("client_secret")
redirect_uri = os.environ.get("redirect_uri")
scope = "user-read-private playlist-modify-public"

df = pd.read_csv("My Thumbs Up.csv")
#http://localhost/?code=AQC9lIGOCwdtw3QLXi9pcWiKvhj4d_NiKukntMirgDwwKFu_2fNGBqhkqaBA7jxftJR5GNlTfUbDPC1yddJQjXozrjj1GZCqiM2QIBBbs3LgbwkzeJi0IK8TqFF1lG0ZvWK6udy0FODOIo9NI_ucK-QBoas&state=playlist-modify-public

In [65]:
import requests
import base64
import random
import string


# Generate a random code verifier and a base64-encoded code challenge
code_verifier = "".join(random.choice(string.ascii_letters + string.digits) for _ in range(64))
code_challenge = base64.urlsafe_b64encode(code_verifier.encode()).decode().replace("=", "")

# Request user authorization by redirecting them to the /authorize endpoint with the required parameters
auth_url = "https://accounts.spotify.com/authorize"
auth_params = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": redirect_uri,
    "state": code_challenge,
    "scope": scope
}
response = requests.get(auth_url, params=auth_params)
print(response.url) # This is the URL that the user should visit to grant permission

# Retrieve the authorization code from the redirect URI after the user grants permission
# For example, if the redirect URI is http://localhost:8080/callback, then the code will be in the query parameter "code"
# You can use the requests library to parse the query parameters from the redirect URI
# redirect_response = requests.get(redirect_uri)


https://accounts.spotify.com/en/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fscope%3Duser-read-private%2Bplaylist-modify-public%26response_type%3Dcode%26redirect_uri%3Dhttp%253A%252F%252Flocalhost%253A8080%26state%3DNDdTNnRhT1FHQzhHU3ROYTdMSW1Sc3Jnb01jY0tES0ZtTmVHTWZsaXRwd1BzUUFoV1A0b2kwdXFPRjVNREpEaA%26client_id%3Dcdfcb8e2b36e4f24b7c37b546eccd354


In [66]:
code = "AQBXdFJVgOCZqzwLOGUeE0t0LyDQasQL8QNHyNEB2WC1Z8b7kmp-dwaD43-NLIShlhnhSfX9p8o3FtGaZZVXhxCPy7jUcjtbGQYTc9OSwAt_85eHZrZYO7z0AdIlUVTfX73MtSva3St7Rd9q0wtCNkFmwfCyO2vBh403wrkLIRYFPX9T5W0nkD6LXr1aBrk1yjjynVyAPfuOJkcEU8K71nn578w"
state = "NDdTNnRhT1FHQzhHU3ROYTdMSW1Sc3Jnb01jY0tES0ZtTmVHTWZsaXRwd1BzUUFoV1A0b2kwdXFPRjVNREpEaA"

# Exchange the authorization code for an access token and a refresh token by sending a POST request to the /token endpoint with the required parameters
token_url = "https://accounts.spotify.com/api/token"
token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": redirect_uri,    
}
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": "Basic " + base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
}
token_response = requests.post(token_url, data=token_data, headers=headers)
token_response_data = token_response.json()

token_response_data

{'access_token': 'BQBmNqIkY7_sfOiV49JjgtBARGA5DJSKo_Jsc8dCx4lVJTGtpxfqBPbxnqpjBowUE46KW5paO8ZimCccqgRDt8vnAj5ZDEkQ_Zt0WWC206DpeN1RNWRJC6vxS0Y3Ar6sSnkAx98wpKE_rXqAxkQrvSQQjcTdEumYkDTHFwGVpiv9Ah6fTabUrx2u4gxpsNNhEJ3z1SbTgfMXYXRFSHhzRos',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQDNhL3wje6L2ikiv7Z7IJ0yoPOr3oCmXL-UAYdHFnHHEOEmY4Igp_1nLNNXK2ro5-d0ywt9kSzOltkhs2J4x-7U97AtYH1ufigWGBQdyvnsRRuB3Um_9DAi_pXJ9T3xOxg',
 'scope': 'playlist-modify-public user-read-private'}

In [67]:
access_token = token_response_data["access_token"]
refresh_token = token_response_data["refresh_token"]

# Use the access token to make API calls to the Spotify endpoints that you have permission for, by adding the Authorization header to your requests
api_url = "https://api.spotify.com/v1/me/"
api_headers = {"Authorization": f"Bearer {access_token}"}
api_response = requests.get(api_url, headers=api_headers)
api_response_data = api_response.json()
print(api_response_data) # This will print the user's currently playing track and playback state



{'display_name': 'minoseah629', 'external_urls': {'spotify': 'https://open.spotify.com/user/121308190'}, 'href': 'https://api.spotify.com/v1/users/121308190', 'id': '121308190', 'images': [], 'type': 'user', 'uri': 'spotify:user:121308190', 'followers': {'href': None, 'total': 0}, 'country': 'US', 'product': 'free', 'explicit_content': {'filter_enabled': False, 'filter_locked': False}}


In [64]:
# Refresh the access token when it expires by sending a POST request to the /token endpoint with the required parameters
refresh_url = "https://accounts.spotify.com/api/token"
refresh_data = {
    "client_id": client_id,
    "grant_type": "refresh_token",
    "refresh_token": refresh_token
}
refresh_response = requests.post(refresh_url, data=refresh_data)
refresh_response_data = refresh_response.json()
refresh_response_data
# access_token = refresh_response_data["access_token"]
# access_token

{'error': 'invalid_request', 'error_description': ''}

In [79]:
user_id = api_response_data["id"]
# Create a playlist with a name and description
playlist_name = "My Playlist"
playlist_description = "A playlist created by Copilot"
playlist_data = {
    "name": playlist_name,
    "description": playlist_description
}

playlist = requests.post(f"https://api.spotify.com/v1/users/{user_id}/playlists", headers=api_headers, data=json.dumps(playlist_data)).json()
# Loop through the dataframe and search for each track on Spotify

In [51]:
playlist

{'collaborative': False,
 'description': 'A playlist created by Copilot',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0YVVGQOKpeO53OGSUHlvHj'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/0YVVGQOKpeO53OGSUHlvHj',
 'id': '0YVVGQOKpeO53OGSUHlvHj',
 'images': [],
 'name': 'My Playlist',
 'owner': {'display_name': 'minoseah629',
  'external_urls': {'spotify': 'https://open.spotify.com/user/121308190'},
  'href': 'https://api.spotify.com/v1/users/121308190',
  'id': '121308190',
  'type': 'user',
  'uri': 'spotify:user:121308190'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSxlYTQ1YTA0NDdhZThlOTMzOGYzOGFkZmEzZGFkYzJiYTNlZDc3M2Q0',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/0YVVGQOKpeO53OGSUHlvHj/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:0YVVGQOKpeO53OGSUHlvHj'}

In [80]:
# Initialize an empty list of track uris
track_uris = []
# Initialize a counter variable
counter = 0
for index, row in df.iterrows():
    # Construct the query string with the track name and artist
    query = f"track:{row['name']} artist:{row['artist']}"
    # Search for the track on Spotify and get the first result
    response = requests.get(f"https://api.spotify.com/v1/search?q={query}&type=track&limit=1", headers=api_headers)    
    # Check if the result is not empty
    result = response.json()
    if (response.status_code == 200):
        if result["tracks"]["items"]:
            # Get the track id
            track_id = result["tracks"]["items"][0]["id"]
            # Add the track uri to the list
            track_uris.append(f"spotify:track:{track_id}")
            # Increment the counter
            counter += 1
            # Print a confirmation message
            print(f"Added {row['name']} by {row['artist']} to {playlist_name}")
            # Check if the counter is 100
            if counter == 100:
                # Post the list of track uris to the playlist
                requests.post(f"https://api.spotify.com/v1/playlists/{playlist['id']}/tracks", headers=api_headers, data=json.dumps({"uris": track_uris})).json()
                # Reset the list and the counter
                track_uris = []
                counter = 0
        else:
            # Print a warning message
            print(f"Could not find {row['name']} by {row['artist']} on Spotify")
# Post the remaining track uris to the playlist
if track_uris:
    requests.post(f"https://api.spotify.com/v1/playlists/{playlist['id']}/tracks", headers=api_headers, data=json.dumps({"uris": track_uris})).json()


Added Already Over by Sabrina Carpenter to My Playlist
Added Do It All for You by Alan Walker to My Playlist
Could not find I'm Yours (Original Version) by Alessia Cara on Spotify
Added Women's Sexual Fantasies by Neal Brennan to My Playlist
Added Napkins in Asia - Asian American by Jim Gaffigan to My Playlist
Could not find Living With Women by Andrew Schulz on Spotify
Added Lily by Alan Walker, K-391 & Emelie Hollow to My Playlist
Added ray of truth by Ayumi Hamasaki to My Playlist
Added Glad by Tori Kelly to My Playlist
Added Dearest by Ayumi Hamasaki to My Playlist
Could not find Egypt Pyramids by Andrew Schulz on Spotify
Added I Wanna Be With You by Mandy Moore to My Playlist
Added Bam Bam (feat. Ed Sheeran) by Camila Cabello to My Playlist
Added A Little Space by Pentatonix to My Playlist
Added goodnight n go by Ariana Grande to My Playlist
Added Middle Ground (feat. CHIKA) by Alessia Cara to My Playlist
Added Can You Feel the Love Tonight by Pentatonix to My Playlist
Added Mad W

In [78]:
print(counter)
df.iloc[124]

24


id               TR:5478797
name               Teaching
artist    Noah Gardenswartz
album                 Blunt
Name: 124, dtype: object